In [43]:
from __future__ import print_function, division
import sys

import tensorflow as tf
tf.disable_v2_behavior()


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [44]:
from keras.datasets import mnist

from keras.models import Sequential, Model
from keras.layers import (
    Dense,
    Reshape,
    UpSampling2D,
    Conv2D,
    BatchNormalization,
    Activation,
    Input,
    Dropout,
    ZeroPadding2D,
    Flatten
)
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

In [45]:
# Create a model using Generator and Degenerator
# Generator creates fake images
# Degenerator real images

In [46]:
img_rows = 28
img_cols = 28
channels = 1
latent_dim = 100
img_shape = (img_rows, img_cols, channels)

In [47]:
def build_generator():
    
    model = Sequential()
    
    # Dimension of output space = 128 * 7 * 7
    model.add(Dense(128 * 7 * 7, activation='relu', input_dim=latent_dim))

    # Reshapes an output to a 7 * 7 * 128
    model.add(Reshape((7, 7, 128)))

    model.add(UpSampling2D())


    model.add(Conv2D(128, kernel_size=3, padding='same'))

    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Activation('relu'))
    
    model.add(UpSampling2D())
    
    model.add(Conv2D(64, kernel_size=3, padding='same'))

    model.add(BatchNormalization(momentum=0.8))

    model.add(Activation('relu'))

    model.add(Conv2D(channels, kernel_size=1, padding='same'))

    model.add(Activation('tanh'))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)


In [48]:
def build_discriminator():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))

    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))

    model.add(ZeroPadding2D(padding=((0,1),(0,1))))

    model.add(BatchNormalization(momentum=0.8))

    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))

    model.add(BatchNormalization(momentum=0.8))

    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))

    model.add(BatchNormalization(momentum=0.8))

    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [49]:
optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

generator = build_generator()

# The generator will take noise as input and generate images
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model (attacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

In [54]:
def save_images(epochs):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r*c, latent_dim))
    gen_imgs = generator.predict(noise)
    
    # Rescale Images 0-1
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r,c)
    
    cnt = 0
    for i in range (r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig(f'images/mnist_{epochs}.png')
    plt.close()

In [55]:
def train(epochs, batch_size=128, save_interval=50):
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    
    # Rescale -1 to 1
    
    X_train = X_train / 127.5 - 1
    X_train = np.expand_dims(X_train, axis=3)
    
    # Adversial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        
        # Train discriminator
        
        # Select random half of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        
        # Sample noise and generate batch of new images
        
        noise = np.random.normal(0,1,(batch_size,latent_dim))
        gen_imgs = generator.predict(noise)
        
        # Train the discriminitor
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss  = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        
        # Train generator
        g_loss = combined.train_on_batch(noise, valid)
        
        print (f"{epoch} [D loss: {d_loss[0]}, acc: {100*d_loss[1]} ] [G loss: {g_loss}]")
        
        if epoch % save_interval == 0:
            save_images(epoch)

In [56]:
!mkdir -p images

In [57]:
train(epochs=4000, batch_size=32, save_interval=50)

/home/ishan/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.6511446237564087, acc: 59.375 ] [G loss: 0.7743724584579468]
1 [D loss: 0.6290213465690613, acc: 71.875 ] [G loss: 0.8939676284790039]
2 [D loss: 0.5867213606834412, acc: 68.75 ] [G loss: 0.739840030670166]
3 [D loss: 0.610511302947998, acc: 70.3125 ] [G loss: 0.7644134759902954]
4 [D loss: 0.4791654050350189, acc: 73.4375 ] [G loss: 0.6300113201141357]
5 [D loss: 0.3733052611351013, acc: 87.5 ] [G loss: 0.6024433374404907]
6 [D loss: 0.4310656487941742, acc: 79.6875 ] [G loss: 0.7397854328155518]
7 [D loss: 0.515370786190033, acc: 73.4375 ] [G loss: 0.699958086013794]
8 [D loss: 0.6042486429214478, acc: 67.1875 ] [G loss: 0.8259195685386658]
9 [D loss: 0.7750558257102966, acc: 57.8125 ] [G loss: 1.0156376361846924]
10 [D loss: 0.7822701930999756, acc: 53.125 ] [G loss: 1.3522310256958008]
11 [D loss: 0.6501046419143677, acc: 65.625 ] [G loss: 1.4551101922988892]
12 [D loss: 0.650460422039032, acc: 68.75 ] [G loss: 1.213249683380127]
13 [D loss: 0.7672992944717407, acc: 64

111 [D loss: 0.8292728662490845, acc: 40.625 ] [G loss: 0.8475192785263062]
112 [D loss: 0.6745136380195618, acc: 60.9375 ] [G loss: 1.0020407438278198]
113 [D loss: 0.7010375261306763, acc: 59.375 ] [G loss: 0.89018714427948]
114 [D loss: 0.8520654439926147, acc: 46.875 ] [G loss: 0.9286243915557861]
115 [D loss: 0.6554532051086426, acc: 64.0625 ] [G loss: 0.9362192153930664]
116 [D loss: 0.8620358109474182, acc: 39.0625 ] [G loss: 0.8707926273345947]
117 [D loss: 0.7052369117736816, acc: 57.8125 ] [G loss: 0.8916658163070679]
118 [D loss: 0.7741700410842896, acc: 48.4375 ] [G loss: 0.871305525302887]
119 [D loss: 0.8965704441070557, acc: 39.0625 ] [G loss: 0.9867116212844849]
120 [D loss: 0.7483854293823242, acc: 51.5625 ] [G loss: 1.0527881383895874]
121 [D loss: 0.735015332698822, acc: 53.125 ] [G loss: 0.9689071774482727]
122 [D loss: 0.8942584991455078, acc: 39.0625 ] [G loss: 1.000124454498291]
123 [D loss: 0.7258723974227905, acc: 65.625 ] [G loss: 1.014451026916504]
124 [D los

221 [D loss: 0.7222335338592529, acc: 50.0 ] [G loss: 0.9012511372566223]
222 [D loss: 0.808292031288147, acc: 46.875 ] [G loss: 0.9002771377563477]
223 [D loss: 0.7495734691619873, acc: 45.3125 ] [G loss: 0.9081453084945679]
224 [D loss: 0.8436317443847656, acc: 35.9375 ] [G loss: 0.8512823581695557]
225 [D loss: 0.9072625041007996, acc: 31.25 ] [G loss: 0.8317937850952148]
226 [D loss: 0.7258685827255249, acc: 53.125 ] [G loss: 0.8711789846420288]
227 [D loss: 0.7381000518798828, acc: 46.875 ] [G loss: 0.842265784740448]
228 [D loss: 0.7407082319259644, acc: 51.5625 ] [G loss: 0.9299547672271729]
229 [D loss: 0.7250916957855225, acc: 56.25 ] [G loss: 0.7092345952987671]
230 [D loss: 0.6866145133972168, acc: 53.125 ] [G loss: 0.6766300797462463]
231 [D loss: 0.7970701456069946, acc: 48.4375 ] [G loss: 0.9397618770599365]
232 [D loss: 0.707457959651947, acc: 56.25 ] [G loss: 0.9964728355407715]
233 [D loss: 0.7330145835876465, acc: 56.25 ] [G loss: 0.8836266994476318]
234 [D loss: 0.72

331 [D loss: 0.742309033870697, acc: 42.1875 ] [G loss: 0.9271062016487122]
332 [D loss: 0.7025970816612244, acc: 56.25 ] [G loss: 0.7997742891311646]
333 [D loss: 0.7203915119171143, acc: 53.125 ] [G loss: 0.7725191116333008]
334 [D loss: 0.774419903755188, acc: 43.75 ] [G loss: 0.8800573348999023]
335 [D loss: 0.7389716506004333, acc: 50.0 ] [G loss: 0.8162286281585693]
336 [D loss: 0.6826364994049072, acc: 56.25 ] [G loss: 0.7974075078964233]
337 [D loss: 0.6715734004974365, acc: 54.6875 ] [G loss: 0.8327100276947021]
338 [D loss: 0.7501449584960938, acc: 51.5625 ] [G loss: 0.88329017162323]
339 [D loss: 0.6909859776496887, acc: 56.25 ] [G loss: 0.889633059501648]
340 [D loss: 0.8216037750244141, acc: 39.0625 ] [G loss: 0.7436449527740479]
341 [D loss: 0.8091940879821777, acc: 43.75 ] [G loss: 0.7528470754623413]
342 [D loss: 0.760310173034668, acc: 45.3125 ] [G loss: 0.975178062915802]
343 [D loss: 0.7199633717536926, acc: 53.125 ] [G loss: 0.9140123128890991]
344 [D loss: 0.776301

...

3558 [D loss: 0.521109402179718, acc: 81.25 ] [G loss: 1.104478120803833]
3559 [D loss: 0.58122318983078, acc: 64.0625 ] [G loss: 1.046494483947754]
3560 [D loss: 0.561302900314331, acc: 75.0 ] [G loss: 1.0113275051116943]
3561 [D loss: 0.54291832447052, acc: 78.125 ] [G loss: 0.9890087246894836]
3562 [D loss: 0.5020611882209778, acc: 85.9375 ] [G loss: 1.001666784286499]
3563 [D loss: 0.5724866390228271, acc: 78.125 ] [G loss: 1.1328833103179932]
3564 [D loss: 0.5627577304840088, acc: 71.875 ] [G loss: 0.998403012752533]
3565 [D loss: 0.5735411047935486, acc: 75.0 ] [G loss: 1.0218260288238525]
3566 [D loss: 0.6126530170440674, acc: 70.3125 ] [G loss: 0.7461955547332764]
3567 [D loss: 0.5155212879180908, acc: 75.0 ] [G loss: 1.2285711765289307]
3568 [D loss: 0.5458840727806091, acc: 78.125 ] [G loss: 1.0518441200256348]
3569 [D loss: 0.7488861680030823, acc: 54.6875 ] [G loss: 0.9095770120620728]
3570 [D loss: 0.6078993678092957, acc: 70.3125 ] [G loss: 0.7376924157142639]
3571 [D los

3665 [D loss: 0.5834718942642212, acc: 75.0 ] [G loss: 1.1984577178955078]
3666 [D loss: 0.5907813906669617, acc: 62.5 ] [G loss: 1.3125379085540771]
3667 [D loss: 0.6149990558624268, acc: 64.0625 ] [G loss: 0.7311037182807922]
3668 [D loss: 0.6008280515670776, acc: 70.3125 ] [G loss: 1.1662297248840332]
3669 [D loss: 0.69264155626297, acc: 57.8125 ] [G loss: 0.9707606434822083]
3670 [D loss: 0.6086375713348389, acc: 73.4375 ] [G loss: 0.9348581433296204]
3671 [D loss: 0.5356380343437195, acc: 76.5625 ] [G loss: 1.150404691696167]
3672 [D loss: 0.6032133102416992, acc: 62.5 ] [G loss: 0.9981815814971924]
3673 [D loss: 0.5590418577194214, acc: 75.0 ] [G loss: 1.0156033039093018]
3674 [D loss: 0.5980850458145142, acc: 62.5 ] [G loss: 0.8895678520202637]
3675 [D loss: 0.6324734687805176, acc: 67.1875 ] [G loss: 0.827397346496582]
3676 [D loss: 0.5052295327186584, acc: 84.375 ] [G loss: 1.1173193454742432]
3677 [D loss: 0.5387969613075256, acc: 71.875 ] [G loss: 1.185647964477539]
3678 [D 

3772 [D loss: 0.5642527937889099, acc: 75.0 ] [G loss: 1.0344364643096924]
3773 [D loss: 0.6875218152999878, acc: 59.375 ] [G loss: 0.9995253086090088]
3774 [D loss: 0.5173858404159546, acc: 82.8125 ] [G loss: 0.9869775772094727]
3775 [D loss: 0.599410891532898, acc: 65.625 ] [G loss: 1.008935809135437]
3776 [D loss: 0.6295193433761597, acc: 62.5 ] [G loss: 1.035445213317871]
3777 [D loss: 0.637645959854126, acc: 60.9375 ] [G loss: 0.859211266040802]
3778 [D loss: 0.6693081259727478, acc: 57.8125 ] [G loss: 0.9550641179084778]
3779 [D loss: 0.5422903299331665, acc: 76.5625 ] [G loss: 0.9839491844177246]
3780 [D loss: 0.6898418664932251, acc: 59.375 ] [G loss: 1.2731554508209229]
3781 [D loss: 0.665535032749176, acc: 56.25 ] [G loss: 1.011399745941162]
3782 [D loss: 0.6449707746505737, acc: 62.5 ] [G loss: 0.9974834322929382]
3783 [D loss: 0.5934629440307617, acc: 67.1875 ] [G loss: 1.086336374282837]
3784 [D loss: 0.6495634317398071, acc: 57.8125 ] [G loss: 1.0029070377349854]
3785 [D 

3879 [D loss: 0.7535908222198486, acc: 53.125 ] [G loss: 1.0539898872375488]
3880 [D loss: 0.5514867901802063, acc: 78.125 ] [G loss: 1.129997968673706]
3881 [D loss: 0.4858272671699524, acc: 81.25 ] [G loss: 1.153365135192871]
3882 [D loss: 0.4821398854255676, acc: 84.375 ] [G loss: 1.0486255884170532]
3883 [D loss: 0.6474608778953552, acc: 53.125 ] [G loss: 1.0344650745391846]
3884 [D loss: 0.6285198926925659, acc: 67.1875 ] [G loss: 0.9167174100875854]
3885 [D loss: 0.6073808670043945, acc: 65.625 ] [G loss: 1.0301004648208618]
3886 [D loss: 0.5076019763946533, acc: 79.6875 ] [G loss: 1.2206065654754639]
3887 [D loss: 0.6454699039459229, acc: 62.5 ] [G loss: 1.0003389120101929]
3888 [D loss: 0.5343101620674133, acc: 76.5625 ] [G loss: 0.8866145610809326]
3889 [D loss: 0.6009812951087952, acc: 71.875 ] [G loss: 0.8132399320602417]
3890 [D loss: 0.6817986965179443, acc: 64.0625 ] [G loss: 0.9298830032348633]
3891 [D loss: 0.6427502632141113, acc: 60.9375 ] [G loss: 1.2107315063476562]

3986 [D loss: 0.5529570579528809, acc: 73.4375 ] [G loss: 0.9260500073432922]
3987 [D loss: 0.6427760124206543, acc: 56.25 ] [G loss: 1.0884840488433838]
3988 [D loss: 0.6121728420257568, acc: 62.5 ] [G loss: 0.8842906355857849]
3989 [D loss: 0.6337028741836548, acc: 59.375 ] [G loss: 1.2211520671844482]
3990 [D loss: 0.655207097530365, acc: 59.375 ] [G loss: 0.9453886151313782]
3991 [D loss: 0.5753989815711975, acc: 71.875 ] [G loss: 1.2372008562088013]
3992 [D loss: 0.7048152089118958, acc: 56.25 ] [G loss: 0.8977617025375366]
3993 [D loss: 0.5994989275932312, acc: 68.75 ] [G loss: 1.1099802255630493]
3994 [D loss: 0.6012268662452698, acc: 71.875 ] [G loss: 1.1618852615356445]
3995 [D loss: 0.6017032861709595, acc: 67.1875 ] [G loss: 1.1035866737365723]
3996 [D loss: 0.550162136554718, acc: 73.4375 ] [G loss: 0.967233419418335]
3997 [D loss: 0.5895019769668579, acc: 71.875 ] [G loss: 1.0553350448608398]
3998 [D loss: 0.5730279684066772, acc: 70.3125 ] [G loss: 0.990531861782074]
3999

In [58]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))



In [59]:
sess

In [60]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4239436313679398901
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10096402140668112227
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8658473216729369118
physical_device_desc: "device: XLA_GPU device"
]


In [61]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [62]:
import keras
import tensorflow as tf


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 16} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [63]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]